In [7]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)

from weighted_accuracy_and_tools import decompose_y, reconstruct_y, weighted_accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd

In [8]:
X_path= os.path.join("..", "..", "data","enriched_input", "X_train.csv")
X = pd.read_csv(X_path, delimiter=',')

y_path= os.path.join("..", "..", "data","enriched_input", "y_train.csv")
y = pd.read_csv(y_path, delimiter=',')

In [9]:
X.set_index("DELIVERY_START", inplace=True)
y.set_index("DELIVERY_START", inplace=True)
X.index = pd.to_datetime(X.index, utc=True)
y.index = pd.to_datetime(y.index, utc=True)

In [10]:
X

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,cold_rate,av_consuption_hour,hour,dayofweek,quarter,month,year,dayofyear
DELIVERY_START,,,,,,,,,,,,,,,,
2022-01-01 01:00:00+00:00,-0.405599,1.143744,0.049156,1.145111,-0.441788,-0.709854,-0.349637,-0.580887,-1.112973,-0.775054,-1.519140,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 02:00:00+00:00,-0.682678,1.143744,0.049156,1.145111,-0.409506,-0.709854,-0.482002,-0.580887,-1.112973,-1.010691,-1.374581,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 03:00:00+00:00,-0.810713,1.143744,0.049156,1.145111,-0.366164,-0.709854,-0.614471,-0.580887,-1.112973,-1.565537,-1.230023,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 04:00:00+00:00,-0.846578,1.143744,0.049156,1.145111,-0.318637,-0.709854,-0.676317,-0.580887,-1.112973,-1.991068,-1.085464,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
2022-01-01 05:00:00+00:00,-0.798790,1.143744,0.049156,1.145111,-0.249289,-0.709854,-0.715292,-0.580887,-1.112973,-2.026920,-0.940906,1.002602,-1.045267,-1.287117,-0.494838,-1.406310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 17:00:00+00:00,-0.275482,1.143744,1.010389,0.384129,0.908396,-0.469079,0.924332,-0.399128,-0.492362,-0.085946,0.793798,-0.489343,-1.045267,-0.734099,2.020864,-0.618926
2023-03-29 18:00:00+00:00,-0.293084,1.143744,1.010389,0.384129,1.143341,-0.669539,0.230275,-0.522000,-0.492362,0.271836,0.938356,-0.489343,-1.045267,-0.734099,2.020864,-0.618926
2023-03-29 19:00:00+00:00,-0.522752,1.143744,1.010389,0.384129,1.375595,-0.709854,0.011252,-0.580887,-0.492362,0.969350,1.082915,-0.489343,-1.045267,-0.734099,2.020864,-0.618926


In [11]:
y_direction, y_magnitude = decompose_y(y['spot_id_delta'])

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np

# Artificially create feature names for demonstration
feature_names = [f"feature_{i}" for i in range(X.shape[1])]

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_magnitude, test_size=0.2, random_state=42)

# Define a pipeline that includes Sequential Feature Selector and Linear Regression
sfs_linear_pipeline = Pipeline([
    ('sfs', SequentialFeatureSelector(LinearRegression(), n_features_to_select=10, direction='forward')),
    ('reg', LinearRegression())
])

# Train the pipeline on the training data
sfs_linear_pipeline.fit(X_train, y_train)

# Extract the support of the selected features
selected_features = sfs_linear_pipeline.named_steps['sfs'].get_support()

# Extract the names of the selected features
selected_feature_names = [feature_names[i] for i in range(len(selected_features)) if selected_features[i]]

# Create the voting regressor with all models, including our pipeline
voting_reg = VotingRegressor(
    estimators=[
        ('sfs_linear_pipeline', sfs_linear_pipeline),
        ('decision_tree_reg', DecisionTreeRegressor(random_state=42)),
        ('random_forest_reg', RandomForestRegressor(random_state=42)),
        ('gradient_boosting_reg', GradientBoostingRegressor(random_state=42)),
        ('knn_reg', KNeighborsRegressor())
    ]
)

# Train the voting regressor on the original dataset
voting_reg.fit(X_train, y_train)

# Make predictions and evaluate the voting regressor
y_pred_voting = voting_reg.predict(X_test)
rmse_voting = np.sqrt(mean_squared_error(y_test, y_pred_voting))

rmse_voting, selected_feature_names


(17.249830788846133,
 ['feature_0',
  'feature_1',
  'feature_2',
  'feature_4',
  'feature_6',
  'feature_8',
  'feature_9',
  'feature_10',
  'feature_13',
  'feature_15'])

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Assuming X is a DataFrame and y_direction is your target variable
X_train, X_test, y_train, y_test = train_test_split(X, y_direction, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Select features based on importance
selector = SelectFromModel(rf, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Get the mask of selected features
selected_features_mask = selector.get_support()

# Get the names of the selected features
selected_feature_names = X_train.columns[selected_features_mask].tolist()

print("Selected features:", selected_feature_names)

# Re-instantiate the individual classifiers
knn = KNeighborsClassifier(n_neighbors=5)
dtree = DecisionTreeClassifier(random_state=42)
logreg = LogisticRegression(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)

# Create the extended voting classifier with all models, using selected features
extended_voting_clf = VotingClassifier(
    estimators=[
        ('knn', knn),
        ('dtree', dtree),
        ('logreg', logreg),
        ('random_forest', random_forest),
        ('gradient_boosting', gradient_boosting)
    ],
    voting='hard'
)

# Train the extended voting classifier on scaled data
extended_voting_clf.fit(X_train[selected_feature_names], y_train)

# Make predictions and evaluate the extended model
y_pred = extended_voting_clf.predict(X_test[selected_feature_names])

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the voting classifier on selected features:", accuracy)


c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Selected features: ['load_forecast', 'nucelear_power_available', 'wind_power_forecasts_average', 'wind_power_forecasts_std', 'hour', 'dayofweek', 'dayofyear']
Accuracy of the voting classifier on selected features: 0.7868929750117869


Selecting features for rd forest but with original data

In [10]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)

from weighted_accuracy_and_tools import decompose_y, reconstruct_y, weighted_accuracy_score
from process_data import process_data
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd

In [11]:
X_path= os.path.join("..", "..", "data","original_input", "X_train_Wwou3IE.csv")
X_preprocessed = pd.read_csv(X_path, delimiter=',')
y_path= os.path.join("..", "..", "data","original_input", "y_train_jJtXgMX.csv")
y_preprocessed = pd.read_csv(y_path, delimiter=',')

In [12]:
X = process_data(X_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
y = process_data(y_preprocessed.copy(deep=True), None, None, None)

Let's drop 1, 7, 5, 2 and keep 0, 3, 4 and 6

In [13]:
X
#'feature_0', 'feature_3', 'feature_4', 'feature_6'
# feature_4: 0.1946 feature_3: 0.1742 feature_6: 0.1740 feature_0: 0.1610

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std
DELIVERY_START,,,,,,,,
2022-01-01 01:00:00+00:00,-0.405599,1.143744,0.049156,1.145111,-0.441788,-0.709854,-0.349637,-0.580887
2022-01-01 02:00:00+00:00,-0.682678,1.143744,0.049156,1.145111,-0.409506,-0.709854,-0.482002,-0.580887
2022-01-01 03:00:00+00:00,-0.810713,1.143744,0.049156,1.145111,-0.366164,-0.709854,-0.614471,-0.580887
2022-01-01 04:00:00+00:00,-0.846578,1.143744,0.049156,1.145111,-0.318637,-0.709854,-0.676317,-0.580887
2022-01-01 05:00:00+00:00,-0.798790,1.143744,0.049156,1.145111,-0.249289,-0.709854,-0.715292,-0.580887
...,...,...,...,...,...,...,...,...
2023-03-29 17:00:00+00:00,-0.275482,1.143744,1.010389,0.384129,0.908396,-0.469079,0.924332,-0.399128
2023-03-29 18:00:00+00:00,-0.293084,1.143744,1.010389,0.384129,1.143341,-0.669539,0.230275,-0.522000
2023-03-29 19:00:00+00:00,-0.522752,1.143744,1.010389,0.384129,1.375595,-0.709854,0.011252,-0.580887


In [14]:
y_direction, y_magnitude = decompose_y(y['spot_id_delta'])

In [16]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming X, y_magnitude are your features and target variables
X_train, X_test, y_train, y_test = train_test_split(X, y_magnitude, test_size=0.2, random_state=42)

# Use RandomForestRegressor for feature importance assessment
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# Use SelectFromModel to select features based on importance
selector = SelectFromModel(rf, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Get the mask of selected features and their names
selected_features_mask = selector.get_support()
feature_names = [f"feature_{i}" for i in range(X.shape[1])]
selected_feature_names = [feature_names[i] for i in range(len(selected_features_mask)) if selected_features_mask[i]]

# Train a new RandomForestRegressor on the selected features
rf_selected = RandomForestRegressor(random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Make predictions and evaluate the model
y_pred_selected = rf_selected.predict(X_test_selected)
rmse_selected = np.sqrt(mean_squared_error(y_test, y_pred_selected))

print("RMSE with selected features:", rmse_selected)
print("Selected feature names:", selected_feature_names)


c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


RMSE with selected features: 19.270135466666048
Selected feature names: ['feature_0', 'feature_3', 'feature_4', 'feature_6']


In [15]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
import numpy as np

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_direction, test_size=0.2, random_state=42)

# Use RandomForestClassifier for feature importance assessment
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

# Use SelectFromModel to select features based on importance
selector = SelectFromModel(rf_clf, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Get the mask of selected features
selected_features_mask = selector.get_support()

# Assuming we know the feature names, let's generate them for this synthetic dataset
feature_names = [f"feature_{i}" for i in range(X.shape[1])]
# Extract the names of the selected features
selected_feature_names = [feature_names[i] for i in range(len(selected_features_mask)) if selected_features_mask[i]]

# Train a new RandomForestClassifier on the selected features
rf_clf_selected = RandomForestClassifier(random_state=42)
rf_clf_selected.fit(X_train_selected, y_train)

# Make predictions and evaluate the model
y_pred_selected = rf_clf_selected.predict(X_test_selected)
accuracy_selected = accuracy_score(y_test, y_pred_selected)

print("Accuracy with selected features:", accuracy_selected)
print("Selected feature names:", selected_feature_names)


c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Accuracy with selected features: 0.694012258368694
Selected feature names: ['feature_0', 'feature_3', 'feature_4', 'feature_6']


In [18]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import mutual_info_classif, SelectKBest
import numpy as np
import pandas as pd

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_direction, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_importances = rf.feature_importances_

# DecisionTreeClassifier for feature importances
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_importances = dt.feature_importances_

# Mutual Information for feature relevance
mi = mutual_info_classif(X_train, y_train)
mi_normalized = mi / np.max(mi)  # Normalize for comparison

# Combine the importances/relevance scores
combined_importances = (rf_importances + dt_importances + mi_normalized) / 3

# Rank features and select the top ones
num_top_features = 10  # Number of top features to select
top_feature_indices = np.argsort(combined_importances)[-num_top_features:]

# Assuming feature names are like 'feature_0', 'feature_1', etc.
feature_names = [f"feature_{i}" for i in range(X.shape[1])]
top_feature_names_scores = [(feature_names[i], combined_importances[i]) for i in top_feature_indices]

# Print top feature names with their scores
for name, score in sorted(top_feature_names_scores, key=lambda x: x[1], reverse=True):
    print(f"{name}: {score:.4f}")

feature_3: 0.4556
feature_4: 0.1405
feature_6: 0.1168
feature_0: 0.1156
feature_2: 0.1081
feature_5: 0.0606
feature_7: 0.0571
feature_1: 0.0305


In [19]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import pandas as pd

# Assuming X, y_direction are your dataset and target
X_train, X_test, y_train, y_test = train_test_split(X, y_direction, test_size=0.2, random_state=42)

# Initialize models
dtree = DecisionTreeClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)

# Fit models
dtree.fit(X_train, y_train)
random_forest.fit(X_train, y_train)
gradient_boosting.fit(X_train, y_train)

# Get feature importances from tree-based models
dt_importances = dtree.feature_importances_
rf_importances = random_forest.feature_importances_
gb_importances = gradient_boosting.feature_importances_

# Calculate mutual information
mi = mutual_info_classif(X_train, y_train)
mi_normalized = mi / np.max(mi)  # Normalize for comparison

# Average the importances/relevance scores including MI for all models
combined_importances = (dt_importances + rf_importances + gb_importances + mi_normalized) / 4

# Rank features and select the top ones
num_top_features = 10  # Number of top features to select
top_feature_indices = np.argsort(combined_importances)[-num_top_features:]

# Assuming feature names are defined as follows
feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]
top_feature_names_scores = [(feature_names[i], combined_importances[i]) for i in top_feature_indices]

# Print top feature names with their scores
for name, score in sorted(top_feature_names_scores, key=lambda x: x[1], reverse=True):
    print(f"{name}: {score:.4f}")


feature_3: 0.4202
feature_4: 0.1508
feature_2: 0.1250
feature_6: 0.1131
feature_0: 0.1118
feature_7: 0.0578
feature_5: 0.0561
feature_1: 0.0259


In [20]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

# Assuming X, y_direction are your dataset and target
X_train, X_test, y_train, y_test = train_test_split(X, y_direction, test_size=0.2, random_state=42)

# Initialize and fit RandomForestClassifier
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)

# Get feature importances
rf_importances = random_forest.feature_importances_

# Rank features based on importance
top_feature_indices = np.argsort(rf_importances)[::-1]

# Assuming feature names are defined
feature_names = [f"feature_{i}" for i in range(X_train.shape[1])]
top_feature_names_scores = [(feature_names[i], rf_importances[i]) for i in top_feature_indices]

# Print top feature names with their scores
print("Feature ranking based on RandomForestClassifier:")
for name, score in top_feature_names_scores:
    print(f"{name}: {score:.4f}")


Feature ranking based on RandomForestClassifier:
feature_4: 0.1946
feature_3: 0.1742
feature_6: 0.1740
feature_0: 0.1610
feature_5: 0.0922
feature_7: 0.0892
feature_2: 0.0839
feature_1: 0.0309


**Feature ranking based on RandomForestClassifier :**

- feature_4: 0.1946
- feature_3: 0.1742
- feature_6: 0.1740
- feature_0: 0.1610
- feature_5: 0.0922
- feature_7: 0.0892
- feature_2: 0.0839
- feature_1: 0.0309

**DT, RF, GB :**
- feature_3: 0.4202
- feature_4: 0.1508
- feature_2: 0.1250
- feature_6: 0.1131
- feature_0: 0.1118
- feature_7: 0.0578
- feature_5: 0.0561
- feature_1: 0.0259

**DT, RF :**
- feature_3: 0.4556
- feature_4: 0.1405
- feature_6: 0.1168
- feature_0: 0.1156
- feature_2: 0.1081
- feature_5: 0.0606
- feature_7: 0.0571
- feature_1: 0.0305

Let's drop 1, 7, 5, 2 and keep 0, 3, 4 and 6

In [30]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# Assuming X, y_direction are your dataset and target
X_train, X_test, y_train, y_test = train_test_split(X, y_magnitude, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
}

# Initialize a DataFrame to store feature importances
features = [f"Feature {i}" for i in range(X.shape[1])]
importances_df = pd.DataFrame(index=features)

# Train each model and collect their feature importances
for name, model in models.items():
    model.fit(X_train, y_train)
    
    # For linear models, use absolute coefficients as a proxy for importance
    if hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)
    elif hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = np.zeros(X.shape[1])  # Placeholder for models without feature importances
    
    importances_df[name] = importances

# Calculate the average importance across all models
importances_df['Average'] = importances_df.mean(axis=1)
importances_df_sorted = importances_df.sort_values(by='Average', ascending=False)

importances_df_sorted


,Linear Regression,Decision Tree Regressor,Random Forest Regressor,Gradient Boosting Regressor,Average
Feature 3,9.951455,0.415757,0.312523,0.593523,2.818315
Feature 0,7.724871,0.075803,0.125569,0.107122,2.008341
Feature 1,3.073557,0.009414,0.089589,0.014379,0.796735
Feature 4,0.787786,0.258001,0.139293,0.038581,0.305915
Feature 6,0.717280,0.118684,0.127327,0.039058,0.250587
Feature 2,0.644038,0.044772,0.081153,0.106171,0.219033
Feature 5,0.485039,0.043361,0.061420,0.020148,0.152492
Feature 7,0.096829,0.034210,0.063125,0.081017,0.068795


In [31]:
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale feature importances for each model before averaging
for model_name in models.keys():
    # Reshape importances data to fit scaler input requirements and scale
    importances_reshaped = importances_df[model_name].values.reshape(-1, 1)
    scaled_importances = scaler.fit_transform(importances_reshaped).flatten()
    importances_df[f"{model_name} (Scaled)"] = scaled_importances

# Calculate the average of the scaled importances
scaled_columns = [f"{name} (Scaled)" for name in models.keys()]
importances_df['Average (Scaled)'] = importances_df[scaled_columns].mean(axis=1)
importances_df_sorted_scaled = importances_df.sort_values(by='Average (Scaled)', ascending=False)

importances_df_sorted_scaled[['Average (Scaled)'] + scaled_columns]


,Average (Scaled),Linear Regression (Scaled),Decision Tree Regressor (Scaled),Random Forest Regressor (Scaled),Gradient Boosting Regressor (Scaled)
Feature 3,1.000000,1.000000,1.000000,1.000000,1.000000
Feature 0,0.338262,0.774057,0.163382,0.255470,0.160139
Feature 4,0.258449,0.070115,0.611765,0.310125,0.041790
Feature 6,0.159238,0.062960,0.268910,0.262470,0.042613
Feature 1,0.103561,0.302064,0.000000,0.112180,0.000000
Feature 2,0.094906,0.055528,0.087015,0.078583,0.158497
Feature 7,0.045719,0.000000,0.061022,0.006791,0.115063
Feature 5,0.033224,0.039394,0.083542,0.000000,0.009962


In [34]:
random_forest_reg = RandomForestRegressor(random_state=42)
random_forest_reg.fit(X_train, y_train)

# Extract and rank feature importances
feature_importances = random_forest_reg.feature_importances_
features_sorted_indices = np.argsort(feature_importances)[::-1]  # Indices of features sorted by importance

# Use the sorted indices to arrange names and importances correctly
sorted_feature_names = [f"Feature {i}" for i in features_sorted_indices]
sorted_feature_importances = feature_importances[features_sorted_indices]

# Create a DataFrame with sorted values
df_importances_corrected = pd.DataFrame({
    "Feature": sorted_feature_names,
    "Importance": sorted_feature_importances,
})

df_importances_corrected

,Feature,Importance
0,Feature 3,0.312523
1,Feature 4,0.139293
2,Feature 6,0.127327
3,Feature 0,0.125569
4,Feature 1,0.089589
5,Feature 2,0.081153
6,Feature 7,0.063125
7,Feature 5,0.061420


**Random Forest Regressor :**
- 0	Feature 3	0.312523
- 1	Feature 4	0.139293
- 2	Feature 6	0.127327
- 3	Feature 0	0.125569
- 4	Feature 1	0.089589
- 5	Feature 2	0.081153
- 6	Feature 7	0.063125
- 7	Feature 5	0.061420


**Average :**
- Feature 3	1.000000
- Feature 0	0.338262
- Feature 4	0.258449
- Feature 6	0.159238
- Feature 1	0.103561
- Feature 2	0.094906
- Feature 7	0.045719
- Feature 5	0.033224

We drop 5, 7, 2, 1 we keep 3, 4, 0, 6